# Model Building using Quarterly data

In [1]:
import os 
import numpy as np 
import pandas as pd
from shapely import wkt
import geopandas as gpd

os.chdir('/Users/kaushalk/Desktop/open_projects/Telengana_Crop_Health')
# Add the 'src' directory to the Python path
from src.gee_download import main, format_date, process_row
from src.gee_download_monthly import download_s2_monthly
from src.process_data import compute_monthly_indices_for_df, compute_indices_for_df

In [3]:
trainq_path = 'data/s2_images/gee_downloads_train_Qtr/train_s2q_datapath.csv'
testq_path = 'data/s2_images/gee_downloads_test_Qtr/test_s2q_datapath.csv'

In [5]:
train_qtr_df = pd.read_csv(trainq_path)
test_qtr_df = pd.read_csv(testq_path)
train_qtr_df.shape, test_qtr_df.shape

((7888, 28), (2718, 27))

In [42]:
test_qtr_df.head()

,FarmID,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,...,Season,geometry,M1_tif_path,M2_tif_path,M3_tif_path,M4_tif_path,M5_tif_path,M6_tif_path,M7_tif_path,M8_tif_path
0,779677,Paddy,Telangana,Medak,Nizampet,2023-12-20 0:00:00,2024-05-28 0:00:00,91,63,Lentil,...,Rabi,"POLYGON ((78.5469981469516 18.066190017327173,...",data/s2_images/gee_downloads_test_Qtr/M1/mean_...,data/s2_images/gee_downloads_test_Qtr/M2/mean_...,NaN,NaN,NaN,NaN,NaN,NaN
1,1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29 0:00:00,2024-05-18 0:00:00,94,89,Pea,...,Rabi,"POLYGON ((78.55945381903922 18.0801159146244, ...",data/s2_images/gee_downloads_test_Qtr/M1/mean_...,data/s2_images/gee_downloads_test_Qtr/M2/mean_...,NaN,NaN,NaN,NaN,NaN,NaN
2,85197,Paddy,Telangana,Medak,Nizampet,2023-11-12 0:00:00,2024-05-14 0:00:00,81,99,Lentil,...,Rabi,POLYGON ((78.54720514633546 18.063322390283137...,data/s2_images/gee_downloads_test_Qtr/M1/mean_...,data/s2_images/gee_downloads_test_Qtr/M2/mean_...,data/s2_images/gee_downloads_test_Qtr/M3/mean_...,NaN,NaN,NaN,NaN,NaN
3,924784,Paddy,Telangana,Medak,Nizampet,2023-11-16 0:00:00,2024-05-24 0:00:00,93,97,Lentil,...,Rabi,POLYGON ((78.56988121141707 18.092636637053605...,data/s2_images/gee_downloads_test_Qtr/M1/mean_...,data/s2_images/gee_downloads_test_Qtr/M2/mean_...,data/s2_images/gee_downloads_test_Qtr/M3/mean_...,NaN,NaN,NaN,NaN,NaN
4,544841,Paddy,Telangana,Medak,Nizampet,2023-12-30 0:00:00,2024-05-22 0:00:00,87,98,Pea,...,Rabi,POLYGON ((78.56293876041818 18.104647739233755...,data/s2_images/gee_downloads_test_Qtr/M1/mean_...,data/s2_images/gee_downloads_test_Qtr/M2/mean_...,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
#test_qdf = compute_monthly_indices_for_df(test_qtr_df, tif_columns_prefix='M')

In [ ]:
columns_to_drop = [col for col in test_qdf.columns if 'tif_path' in col]
test_qdf = test_qdf.drop(columns=columns_to_drop)
columns_to_fill = [col for col in test_qdf.columns if any(match in col for match in ['M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8'])]
# Fill NaN values with 0 for the selected columns
test_qdf[columns_to_fill] = test_qdf[columns_to_fill].fillna(0)
test_qdf.info()

In [56]:
test_qdf.to_csv('data/processed/input_data/test_qtr_indx.csv')